In [ ]:
!pip install datasets evaluate transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
drive_PATH = '../content/drive/MyDrive/Colab Notebooks/dis.experiments.4'
import sys
sys.path.append(drive_PATH)
# drive_PATH = ''

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import tqdm
import math
import numpy as np
import pandas as pd

In [ ]:
# Device for GPU speedup
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

In [ ]:
### MNLI Dataset ###
!pip install jsonlines
import jsonlines # jsonl imports

train_PATH = drive_PATH + '/data/multinli_1.0/multinli_1.0_train.jsonl'
dev_matched_PATH = drive_PATH + '/data/multinli_1.0/multinli_1.0_dev_matched.jsonl'
dev_mismatched_PATH = drive_PATH + '/data/multinli_1.0/multinli_1.0_dev_mismatched.jsonl'
hans_PATH = drive_PATH + '/data/hans/heuristics_evaluation_set.jsonl'

# Train Data
train_DATA = []
train_s1 = []
train_s2 = []
train_text = []
train_label = []
# Mathced Dev Data
dev_matched_DATA = []
dev_matched_s1 = []
dev_matched_s2 = []
dev_matched_text = []
dev_matched_label = []
# Mismatched Dev Data
dev_mismatched_DATA = []
dev_mismatched_s1 = []
dev_mismatched_s2 = []
dev_mismatched_text = []
dev_mismatched_label = []
# Hans Data
hans_DATA = []
hans_s1 = []
hans_s2 = []
hans_text = []
hans_label = []

with jsonlines.open(train_PATH) as f:
    for line in f.iter():
        train_DATA.append(line)
        train_s1.append(line['sentence1'])
        train_s2.append(line['sentence2'])
        train_text.append( line['sentence1'] + ' ' + line['sentence2'] )
        train_label.append(line['gold_label'])
with jsonlines.open(dev_matched_PATH) as f:
    for line in f.iter():
        dev_matched_DATA.append(line)
        dev_matched_s1.append(line['sentence1'])
        dev_matched_s2.append(line['sentence2'])
        dev_matched_text.append( line['sentence1'] + ' ' + line['sentence2'] )
        dev_matched_label.append(line['gold_label'])
with jsonlines.open(dev_mismatched_PATH) as f:
    for line in f.iter():
        dev_mismatched_DATA.append(line)
        dev_mismatched_s1.append(line['sentence1'])
        dev_mismatched_s2.append(line['sentence2'])
        dev_mismatched_text.append( line['sentence1'] + ' ' + line['sentence2'] )
        dev_mismatched_label.append(line['gold_label'])
with jsonlines.open(hans_PATH) as f:
    for line in f.iter():
        hans_DATA.append(line)
        hans_s1.append(line['sentence1'])
        hans_s2.append(line['sentence2'])
        hans_text.append( line['sentence1'] + ' ' + line['sentence2'] )
        hans_label.append(line['gold_label'])

### Cleaning Datasets

# Train
train_label = np.array(train_label, dtype='<U14')
train_s1 = np.array(train_s1)
train_s2 = np.array(train_s2)
train_label[(train_label == 'neutral') | (train_label == 'contradiction')] = 'non-entailment'
train_label[train_label == ['entailment']] = 1
train_label[train_label == ['non-entailment']] = 0
train_label = np.array(train_label, dtype='int')

# Dev Matched
dev_matched_label = np.array(dev_matched_label, dtype='<U14')
dev_matched_filter = dev_matched_label != '-'
dev_matched_s1 = np.array(dev_matched_s1)[dev_matched_filter]
dev_matched_s2 = np.array(dev_matched_s2)[dev_matched_filter]
dev_matched_label = dev_matched_label[dev_matched_filter]
dev_matched_label[(dev_matched_label == 'neutral') | (dev_matched_label == 'contradiction')] = 'non-entailment'
dev_matched_label[dev_matched_label == ['entailment']] = 1
dev_matched_label[dev_matched_label == ['non-entailment']] = 0
dev_matched_label = np.array(dev_matched_label, dtype='int')

# Dev Mismatched
dev_mismatched_label = np.array(dev_mismatched_label, dtype='<U14')
dev_mismatched_filter = dev_mismatched_label != '-'
dev_mismatched_s1 = np.array(dev_mismatched_s1)[dev_mismatched_filter]
dev_mismatched_s2 = np.array(dev_mismatched_s2)[dev_mismatched_filter]
dev_mismatched_label = dev_mismatched_label[dev_mismatched_filter]
dev_mismatched_label[(dev_mismatched_label == 'neutral') | (dev_mismatched_label == 'contradiction')] = 'non-entailment'
dev_mismatched_label[dev_mismatched_label == ['entailment']] = 1
dev_mismatched_label[dev_mismatched_label == ['non-entailment']] = 0
dev_mismatched_label = np.array(dev_mismatched_label, dtype='int')

# HANS
hans_label = np.array(hans_label)
hans_s1 = np.array(hans_s1)
hans_s2 = np.array(hans_s2)
hans_label[hans_label == ['entailment']] = 1
hans_label[hans_label == ['non-entailment']] = 0
hans_label = np.array(hans_label, dtype='int')

train_labels = np.unique(train_label)
dev_matched_labels = np.unique(dev_matched_label)
dev_mismatched_labels = np.unique(dev_mismatched_label)
hans_labels = np.unique(np.array(hans_label))

value_counts = pd.concat({'train_label' : pd.DataFrame(train_label).value_counts(),
                        'dev_matched_label' : pd.DataFrame(dev_matched_label).value_counts(),
                        'dev_mismatched_label' : pd.DataFrame(dev_mismatched_label).value_counts(),
                        'hans_label' : pd.DataFrame(hans_label).value_counts()})

### Balancing Act
def balanced_idx(label_dataset):
    idx1 = np.array(range(len(label_dataset)))[label_dataset == 1]
    idx0 = np.array(range(len(label_dataset)))[label_dataset == 0]
    idx0_selected_i = np.random.choice(idx0.shape[0], len(idx1), replace=False)
    idx0_selected = idx0[idx0_selected_i]
    idx = np.concatenate((idx1, idx0_selected))
    np.random.shuffle(idx) # random shuffle
    return idx

# Balancing Train
train_balanced_idx = balanced_idx(train_label)
train_s1 = train_s1[train_balanced_idx]
train_s2 = train_s2[train_balanced_idx]
train_label = train_label[train_balanced_idx]

# Balancing Dev Matched
dev_matched_balanced_idx = balanced_idx(dev_matched_label)
dev_matched_s1 = dev_matched_s1[dev_matched_balanced_idx]
dev_matched_s2 = dev_matched_s2[dev_matched_balanced_idx]
dev_matched_label = dev_matched_label[dev_matched_balanced_idx]

# Balancing Dev Mismatched
dev_mismatched_balanced_idx = balanced_idx(dev_mismatched_label)
dev_mismatched_s1 = dev_mismatched_s1[dev_mismatched_balanced_idx]
dev_mismatched_s2 = dev_mismatched_s2[dev_mismatched_balanced_idx]
dev_mismatched_label = dev_mismatched_label[dev_mismatched_balanced_idx]

# Balancing HANS (already balanced)
hans_balanced_idx = balanced_idx(hans_label)
hans_s1 = hans_s1[hans_balanced_idx]
hans_s2 = hans_s2[hans_balanced_idx]
hans_label = hans_label[hans_balanced_idx]

In [ ]:
# ### Calculate maximum sequence length
# from transformers import BertTokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# train_s1_len = np.array([len(tokenizer.encode(s)) for s in tqdm.tqdm(train_s1)])
# train_s2_len = np.array([len(tokenizer.encode(s)) for s in tqdm.tqdm(train_s2)])
# dev_matched_s1_len = np.array([len(tokenizer.encode(s)) for s in tqdm.tqdm(dev_matched_s1)])
# dev_matched_s2_len = np.array([len(tokenizer.encode(s)) for s in tqdm.tqdm(dev_matched_s2)])
# dev_mismatched_s1_len = np.array([len(tokenizer.encode(s)) for s in tqdm.tqdm(dev_mismatched_s1)])
# dev_mismatched_s2_len = np.array([len(tokenizer.encode(s)) for s in tqdm.tqdm(dev_mismatched_s2)])
# hans_s1_len = np.array([len(tokenizer.encode(s)) for s in tqdm.tqdm(hans_s1)])
# hans_s2_len = np.array([len(tokenizer.encode(s)) for s in tqdm.tqdm(hans_s2)])

# ### Maximum
# train_s1_len_max = train_s1_len.max()
# train_s2_len_max = train_s2_len.max()
# dev_matched_s1_len_max = dev_matched_s1_len.max()
# dev_matched_s2_len_max = dev_matched_s2_len.max()
# dev_mismatched_s1_len_max = dev_mismatched_s1_len.max()
# dev_mismatched_s2_len_max = dev_mismatched_s2_len.max()
# hans_s1_len_max = hans_s1_len.max()
# hans_s2_len_max = hans_s2_len.max()
# ### Mean
# train_s1_len_mean = train_s1_len.mean()
# train_s2_len_mean = train_s2_len.mean()
# dev_matched_s1_len_mean = dev_matched_s1_len.mean()
# dev_matched_s2_len_mean = dev_matched_s2_len.mean()
# dev_mismatched_s1_len_mean = dev_mismatched_s1_len.mean()
# dev_mismatched_s2_len_mean = dev_mismatched_s2_len.mean()
# hans_s1_len_mean = hans_s1_len.mean()
# hans_s2_len_mean = hans_s2_len.mean()
# ### Std Deviation
# train_s1_len_std = train_s1_len.std()
# train_s2_len_std = train_s2_len.std()
# dev_matched_s1_len_std = dev_matched_s1_len.std()
# dev_matched_s2_len_std = dev_matched_s2_len.std()
# dev_mismatched_s1_len_std = dev_mismatched_s1_len.std()
# dev_mismatched_s2_len_std = dev_mismatched_s2_len.std()
# hans_s1_len_std = hans_s1_len.std()
# hans_s2_len_std = hans_s2_len.std()

# print(f'Train Sequence Lengths\ns1(max): {train_s1_len_max}\ns2(max): {train_s2_len_max}\ns1(mean): {train_s1_len_mean}\ns2(mean): {train_s2_len_mean}\ns1(std): {train_s1_len_std}\ns2(std): {train_s2_len_std}' )
# print(f'Dev (matched) Sequence Lengths\ns1(max): {dev_matched_s1_len_max}\ns2(max): {dev_matched_s2_len_max}\ns1(mean): {dev_matched_s1_len_mean}\ns2(mean): {dev_matched_s2_len_mean}\ns1(std): {dev_matched_s1_len_std}\ns2(std): {dev_matched_s2_len_std}' )
# print(f'Dev (mismatched) Sequence Lengths\ns1(max): {dev_mismatched_s1_len_max}\ns2(max): {dev_mismatched_s2_len_max}\ns1(mean): {dev_mismatched_s1_len_mean}\ns2(mean): {dev_mismatched_s2_len_mean}\ns1(std): {dev_mismatched_s1_len_std}\ns2(std): {dev_mismatched_s2_len_std}' )
# print(f'HANS Sequence Lengths\ns1(max): {hans_s1_len_max}\ns2(max): {hans_s2_len_max}\ns1(mean): {hans_s1_len_mean}\ns2(mean): {hans_s2_len_mean}\ns1(std): {hans_s1_len_std}\ns2(std): {hans_s2_len_std}' )

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#####################
### Training Data ###
#####################
train_input_ids = []
train_attention_masks = []
train_token_type_ids = []
len_train = len(train_s1)
assert ((len(train_s1) == len_train) and (len(train_s2) == len_train))
for i in tqdm.tqdm(range(len_train)):
  encoded_dict = tokenizer.encode_plus(
                      train_s1[i], train_s2[i],
                      add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                      max_length = 512,           # Pad & truncate all sentences.
                      pad_to_max_length = True,
                      return_attention_mask = True,   # Construct attn. masks.
                      return_tensors = 'pt',     # Return pytorch tensors.
                  )
  train_input_ids.append(encoded_dict['input_ids'])
  train_attention_masks.append(encoded_dict['attention_mask'])
  train_token_type_ids.append(encoded_dict['token_type_ids'])
# Convert the lists into tensors
train_input_ids = torch.cat(train_input_ids, dim=0)
train_attention_masks = torch.cat(train_attention_masks, dim=0)
train_token_type_ids = torch.cat(train_token_type_ids, dim=0)
train_labels = torch.tensor(train_label)

##########################
### Dev (Matched) Data ###
##########################
dev_matched_input_ids = []
dev_matched_attention_masks = []
dev_matched_token_type_ids = []
len_dev_matched = len(dev_matched_s1)
assert ((len(dev_matched_s1) == len_dev_matched) and (len(dev_matched_s2) == len_dev_matched))
for i in tqdm.tqdm(range(len_dev_matched)):
  encoded_dict = tokenizer.encode_plus(
                      dev_matched_s1[i], dev_matched_s2[i],
                      add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                      max_length = 512,           # Pad & truncate all sentences.
                      pad_to_max_length = True,
                      return_attention_mask = True,   # Construct attn. masks.
                      return_tensors = 'pt',     # Return pytorch tensors.
                  )
  dev_matched_input_ids.append(encoded_dict['input_ids'])
  dev_matched_attention_masks.append(encoded_dict['attention_mask'])
  dev_matched_token_type_ids.append(encoded_dict['token_type_ids'])
# Convert the lists into tensors
dev_matched_input_ids = torch.cat(dev_matched_input_ids, dim=0)
dev_matched_attention_masks = torch.cat(dev_matched_attention_masks, dim=0)
dev_matched_token_type_ids = torch.cat(dev_matched_token_type_ids, dim=0)
dev_matched_labels = torch.tensor(dev_matched_label)

#############################
### Dev (Mismatched) Data ###
#############################
dev_mismatched_input_ids = []
dev_mismatched_attention_masks = []
dev_mismatched_token_type_ids = []
len_dev_mismatched = len(dev_mismatched_s1)
assert ((len(dev_mismatched_s1) == len_dev_mismatched) and (len(dev_mismatched_s2) == len_dev_mismatched))
for i in tqdm.tqdm(range(len_dev_mismatched)):
  encoded_dict = tokenizer.encode_plus(
                      dev_mismatched_s1[i], dev_mismatched_s2[i],
                      add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                      max_length = 512,           # Pad & truncate all sentences.
                      pad_to_max_length = True,
                      return_attention_mask = True,   # Construct attn. masks.
                      return_tensors = 'pt',     # Return pytorch tensors.
                  )
  dev_mismatched_input_ids.append(encoded_dict['input_ids'])
  dev_mismatched_attention_masks.append(encoded_dict['attention_mask'])
  dev_mismatched_token_type_ids.append(encoded_dict['token_type_ids'])
# Convert the lists into tensors
dev_mismatched_input_ids = torch.cat(dev_mismatched_input_ids, dim=0)
dev_mismatched_attention_masks = torch.cat(dev_mismatched_attention_masks, dim=0)
dev_mismatched_token_type_ids = torch.cat(dev_mismatched_token_type_ids, dim=0)
dev_mismatched_labels = torch.tensor(dev_mismatched_label)

#################
### HANS Data ###
#################
hans_input_ids = []
hans_attention_masks = []
hans_token_type_ids = []
len_hans = len(hans_s1)
assert ((len(hans_s1) == len_hans) and (len(hans_s2) == len_hans))
for i in tqdm.tqdm(range(len_hans)):
  encoded_dict = tokenizer.encode_plus(
                      hans_s1[i], hans_s2[i],
                      add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                      max_length = 512,           # Pad & truncate all sentences.
                      pad_to_max_length = True,
                      return_attention_mask = True,   # Construct attn. masks.
                      return_tensors = 'pt',     # Return pytorch tensors.
                      )
  hans_input_ids.append(encoded_dict['input_ids'])
  hans_attention_masks.append(encoded_dict['attention_mask'])
  hans_token_type_ids.append(encoded_dict['token_type_ids'])
# Convert the lists into tensors
hans_input_ids = torch.cat(hans_input_ids, dim=0)
hans_attention_masks = torch.cat(hans_attention_masks, dim=0)
hans_token_type_ids = torch.cat(hans_token_type_ids, dim=0)
hans_labels = torch.tensor(hans_label)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

  0%|          | 0/261798 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 30000/30000 [00:25<00:00, 1199.94it/s]


In [ ]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

### Datasets ###
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_token_type_ids, train_labels)
dev_matched_dataset = TensorDataset(dev_matched_input_ids, dev_matched_attention_masks, dev_matched_token_type_ids, dev_matched_labels)
dev_mismatched_dataset = TensorDataset(dev_mismatched_input_ids, dev_mismatched_attention_masks, dev_mismatched_token_type_ids, dev_mismatched_labels)
hans_dataset = TensorDataset(hans_input_ids, hans_attention_masks, hans_token_type_ids , hans_labels)
### Dataset Splits ###
# MNLI
num_train = int(len(train_dataset) * 0.95)
num_valid = len(train_dataset) - num_train #int(len(train_dataset) * 0.05)
split_train, split_valid = random_split( train_dataset, [num_train, num_valid] )
split_test_matched = dev_matched_dataset
split_test_mismatched = dev_mismatched_dataset
# HANS
num_train_hans = int(len(hans_dataset) * 0.80)
num_valid_hans = int(len(hans_dataset) * 0.10)
num_test_hans = len(hans_dataset) - num_train_hans - num_valid_hans #int(len(hans_dataset) * 0.10)
split_train_hans , split_valid_hans, split_test_hans = random_split( hans_dataset, [num_train_hans, num_valid_hans , num_test_hans] )
# Final Datasets
# MNLI
# Training: split_train_ & split_valid_
# Testing: dev_matched_dataset & dev_mismatched_dataset
# HANS
# Training: split_train_train_hans_ & split_train_valid_hans_
# Testing: split_test_hans_

In [ ]:
#########################
#########################
## ---   HERE ---   #####
#########################
#########################

# Todo
# 1. Fix the Loss function and the Backward
# 2. Print and save both the accuracy and Loss
# 3. Also add the token type ids

In [ ]:
##########################
### Define the Model 1 ###
##########################
from transformers import BertForSequenceClassification
import transformers
# Load BertForSequenceClassification
# model = BertForSequenceClassification.from_pretrained(
#     "bert-base-uncased",
#     num_labels = 2, # The number of output labels. 2 for binary classification.
#     output_attentions = False,
#     output_hidden_states = False,
# )
##########################
### Define the Model 2 ###
##########################
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model.
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.l2 = torch.nn.Dropout(0.1)
        self.l3 = torch.nn.Linear(768, 2)

    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        # softmax_output = torch.nn.functional.softmax(output, dim=1)
        return output
model = BERTClass()
model.to(DEVICE)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [ ]:
############################
### Define the Optimizer ###
############################
from transformers import AdamW
LEARNING_RATE =  2e-5
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
################################
### Define the Loss Function ###
################################
loss_function = torch.nn.CrossEntropyLoss()
###############################
### Define the Data Loaders ###
###############################
BATCH_SIZE = 16
# MNLI
train_dataloader = DataLoader(split_train, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(split_valid, batch_size=BATCH_SIZE, shuffle=True)
test_matched_dataloader = DataLoader(split_test_matched, batch_size=BATCH_SIZE, shuffle=True)
test_mismatched_dataloader = DataLoader(split_test_mismatched, batch_size=BATCH_SIZE, shuffle=True)
# HANS
hans_train_dataloader = DataLoader(split_train_hans, batch_size=BATCH_SIZE, shuffle=True)
hans_valid_dataloader = DataLoader(split_valid_hans, batch_size=BATCH_SIZE, shuffle=True)
hans_test_dataloader = DataLoader(split_test_hans, batch_size=BATCH_SIZE, shuffle=True)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
##################################
### Define the Evaluation Loop ###
##################################
def evaluate(dataloader):
    model.cuda()
    model.eval()
    total_loss, total_acc, total_count, total_count_n = 0, 0, 0, 0
    with torch.no_grad():
      for step, batch in tqdm.tqdm(enumerate(dataloader)):
          # 0. Put ids & mask & labels to the device
          b_input_ids = batch[0].to(DEVICE)
          b_input_mask = batch[1].to(DEVICE)
          b_token_type_ids = batch[2].to(DEVICE)
          b_labels = batch[3].to(DEVICE)
          # 2. Forward Pass
          output = model(b_input_ids, b_input_mask, b_token_type_ids)
          logits = output
          predicted_class_labels = logits.argmax(dim=1)
          # 3. Loss
          valid_loss = loss_function(logits, b_labels)
          valid_acc = (predicted_class_labels == b_labels).sum().item()
          total_loss += valid_loss
          total_acc += valid_acc
          total_count += len(b_labels)
          total_count_n += 1
    Loss = total_loss / total_count_n
    Acc = total_acc / total_count
    return Loss, Acc

In [13]:
# TRAINING & VALIDATION
import time
EPOCHS = 4

model_paths = []
# MNLI Losses
train_losses, train_accs = [], []
val_losses, val_accs = [], []
test_matched_losses, test_matched_accs = [], []
test_mismatched_losses, test_mismatched_accs = [], []
# HANS Losses
hans_train_losses , hans_train_accs  = [], []
hans_valid_losses, hans_valid_accs  = [], []
hans_test_losses, hans_test_accs = [], []
# Model ID
model_id = '-'.join(time.ctime(time.time()).replace(':', ' ').split(' ')[2:5])

# DATALOADERS
# MNLI
train_dataloader = DataLoader(split_train, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(split_valid, batch_size=BATCH_SIZE, shuffle=True)
test_matched_dataloader = DataLoader(split_test_matched, batch_size=BATCH_SIZE, shuffle=True)
test_mismatched_dataloader = DataLoader(split_test_mismatched, batch_size=BATCH_SIZE, shuffle=True)
# HANS
hans_train_dataloader = DataLoader(split_train_hans, batch_size=BATCH_SIZE, shuffle=True)
hans_valid_dataloader = DataLoader(split_valid_hans, batch_size=BATCH_SIZE, shuffle=True)
hans_test_dataloader = DataLoader(split_test_hans, batch_size=BATCH_SIZE, shuffle=True)

### EPOCHS ###
for epoch in range(1, EPOCHS + 1):
    ################
    ### TRAINING ###
    ################
    epoch_start_time = time.time()
    ############################################################################
    model.cuda()
    model.train()
    total_loss, total_acc, total_count = 0, 0, 0
    log_interval = 100
    dataloader = train_dataloader # CHANGE TO TRAIN DATALOADER
    for step, batch in tqdm.tqdm(enumerate(dataloader)):
        # 0. Put ids & mask & labels to the device
        b_input_ids = batch[0].to(DEVICE)
        b_input_mask = batch[1].to(DEVICE)
        b_token_type_ids = batch[2].to(DEVICE)
        b_labels = batch[3].to(DEVICE)
        # 1. Clear Gradient
        optimizer.zero_grad()
        # 2. Forward Pass
        output = model(b_input_ids, b_input_mask, b_token_type_ids)
        logits = output                        # predicted probabilities
        predicted_class_labels = logits.argmax(dim=1) # predicted classes
        # 3. Loss
        train_loss = loss_function(logits, b_labels)
        # 4. Backward Pass
        train_loss.backward()
        # 5. Optimizer Step
        optimizer.step()
        # 6. Progress Display
        train_acc = (predicted_class_labels == b_labels).sum().item() / len(b_labels)
        total_loss += train_loss.item()
        total_acc += train_acc
        total_count += 1
        if step % log_interval == 0 and step > 0:
            loss = total_loss / total_count
            acc = total_acc / total_count
            train_losses.append(loss)
            train_accs.append(acc)
            print("| epoch {: d} | {:5d}/{:5d} batches "
                  "| loss {:8.3f}| accuracy {:8.3f}".format(epoch, step, len(dataloader),loss , acc))
            total_loss, total_acc, total_count = 0, 0, 0
    ############################################################################
    epoch_end_time = time.time()
    training_time = epoch_end_time - epoch_start_time
    print("-" * 59)
    print("| end of epoch {:3d} | time: {:5.2f}s |".format( epoch, training_time))
    print("-" * 59)
    ##################
    ### VALIDATION ###
    ##################
    # Training
    loss_train = train_losses[-1]
    acc_train = train_accs[-1]
    # Validation
    loss_val, acc_val = evaluate(valid_dataloader)
    val_losses.append(loss_val)
    val_accs.append(acc_val)
    # Test Matched
    loss_test_matched, acc_test_matched = evaluate(test_matched_dataloader)
    test_matched_losses.append(loss_test_matched)
    test_matched_accs.append(acc_test_matched)
    # Test Mismatched
    loss_test_mismatched, acc_test_mismatched = evaluate(test_mismatched_dataloader)
    test_mismatched_losses.append(loss_test_mismatched)
    test_mismatched_accs.append(acc_test_mismatched)
    # Hans Validation
    loss_hans_valid, acc_hans_valid = evaluate(hans_valid_dataloader)
    hans_valid_losses.append(loss_hans_valid)
    hans_valid_accs.append(acc_hans_valid)
    # Hans Test
    loss_hans_test , acc_hans_test = evaluate(hans_test_dataloader)
    hans_test_losses.append(loss_hans_test)
    hans_test_accs.append(acc_hans_test)
    # Register model path
    model_paths.append(f'id-{model_id}-epoch-{epoch}-acc_train-{acc_train:.2f}-acc_val-{acc_val:.2f}.pt')
    # Save model to path
    torch.save(model.state_dict(), drive_PATH+'/model_states/'+model_paths[-1])
    # Display Results
    print("| Validation Loss {:8.3f} | Validation Accuracy {:8.3f}".format( loss_val, acc_val))
    print("| Test Matched Loss {:8.3f} | Test Matched Accuracy {:8.3f}".format( loss_test_matched, acc_test_matched))
    print("| Test Mismatched Loss {:8.3f} | Test Mismatched Accuracy {:8.3f}".format( loss_test_mismatched, acc_test_mismatched))
    print("| HANS Validation Loss {:8.3f} | HANS Validation Accuracy {:8.3f}".format( loss_hans_valid, acc_hans_valid))
    print("| HANS Test Loss {:8.3f} | HANS Test Accuracy {:8.3f}".format( loss_hans_test, acc_hans_test))
    print("-" * 59)

101it [00:36,  2.83it/s]

| epoch  1 |   100/15545 batches | loss    0.661| accuracy    0.590


201it [01:12,  2.83it/s]

| epoch  1 |   200/15545 batches | loss    0.527| accuracy    0.738


301it [01:47,  2.82it/s]

...

| epoch  3 |  2400/15545 batches | loss    0.116| accuracy    0.958


2501it [14:46,  2.82it/s]

| epoch  3 |  2500/15545 batches | loss    0.112| accuracy    0.956


2524it [14:54,  2.82it/s]


KeyboardInterrupt: 

In [14]:
# Load the model with best validation accuracy
best_model_index = np.argmin([l.cpu() for l in val_losses])
model_state = model_paths[best_model_index]
model.load_state_dict(torch.load(drive_PATH+'/model_states/'+model_state))

<All keys matched successfully>

In [17]:
val_losses

[tensor(0.2862, device='cuda:0'), tensor(0.3208, device='cuda:0')]

In [ ]:
import collections
from functools import partial

NUM_BATCHES = 100
s1_dataset, s2_dataset, labels_dataset = [] , [], []
for idx, (s1, s2, label) in enumerate(valid_dataloader):
	if idx == NUM_BATCHES:
		break
	s1_dataset.append(s1)
	s2_dataset.append(s2)
	labels_dataset.append(label)

# a dictionary that keeps saving the activations as they come
activations = collections.defaultdict(list)
def save_activation(name, mod, inp, out):
	activations[name].append(out.cpu())

# Registering hooks for all the TransformerEncoder layers
# Note: Hooks are called EVERY TIME the module performs a forward pass. For modules that are
# called repeatedly at different stages of the forward pass (like TransformerEncoder in NLI called for s1 and s2 separately)
# this will save different activations.
# Editing the forward pass code to save activations is the way to go for these cases.
# Or we can filter out the odd and even indices from the activations to get the ones for s1 and s2
for name, m in model.named_modules():
	if name == 'encoder':
		# partial to assign the layer name to each hook
		m.register_forward_hook(partial(save_activation, name))
	if name == 'classifier.classifier.1':
		m.register_forward_hook(partial(save_activation, name))
	if name == 'classifier.classifier.4':
		m.register_forward_hook(partial(save_activation, name))
	if name == 'classifier.classifier.7':
		m.register_forward_hook(partial(save_activation, name))

# forward pass through the full dataset
for batch_i in range(NUM_BATCHES):
	out = model(s1_dataset[batch_i], s2_dataset[batch_i])